In [11]:
%load_ext autoreload
%autoreload 2

import os
DATA_DIR = os.path.expanduser("~/repos/data/")
# DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"

FANDE_DIR = os.path.expanduser("~/repos/")
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")
# os.makedirs(RESULTS_DIR, exist_ok=True)

import os
import sys
import torch
sys.path.append(FANDE_DIR + "fande/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
%%time
from ase import io

traj_295 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="0:10")
# traj_355 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")
# traj_295_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
# traj_355_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
# traj_295_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
# traj_355_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

CPU times: user 42.6 ms, sys: 827 µs, total: 43.4 ms
Wall time: 42.1 ms


In [13]:
init_structure = traj_295[0]
init_structure.set_pbc(False)
input_xml_str = """
<simulation verbosity='high'>
  <output prefix='PREFIX'>
<!--    <properties stride='1' filename='out'>  [ step, potential{kilocal/mol} ] </properties>-->
    <!-- <trajectory stride='1' filename='pos'>  positions{angstrom}  </trajectory> -->
    <!-- <properties stride="10"> [ step, time, pot_component_raw(0), pot_component_raw(1), temperature, conserved ] </properties> -->
    <properties stride="1"> [ step, time, potential(0), temperature, conserved ] </properties>
    <trajectory stride="1" filename="pos" cell_units="angstrom"> positions{angstrom} </trajectory>
    <trajectory stride="1" filename="pos_centroid" cell_units="angstrom"> x_centroid{angstrom} </trajectory>
    <trajectory stride="1" filename="frc"> forces </trajectory>
    <checkpoint stride="1000" overwrite="False"/>
  </output>
  <total_steps> 100 </total_steps>
  <ffsocket mode='inet' name='mycalc' pbc='True'>
    <address>localhost</address>
    <port>10200</port>
  </ffsocket>
  <system>
    <initialize nbeads='6'>
      <file mode='ase'>  init.xyz </file>
      <velocities mode="thermal" units="kelvin"> 20 </velocities>
    </initialize>
    <forces>
      <force forcefield='mycalc'> </force>
    </forces>
    <ensemble>
      <temperature units='kelvin'> 300.0 </temperature>
    </ensemble>
    <motion mode='dynamics'>
      <dynamics mode='nvt'>
        <timestep units='femtosecond'> 0.5 </timestep>
        <thermostat mode='langevin'>
          <tau units='femtosecond'>10</tau>
        </thermostat>
      </dynamics>
    </motion>
  </system>
</simulation>
"""

calc_dir = DATA_DIR + "/results/test_run/"
calculator = 'xtb'
num_instances = 6
ipi_port = 10200

In [14]:
from fande import ipi
ipi.kill_all()
ipi.launch(init_structure, calculator, num_instances, calc_dir, input_xml_str, ipi_port)

Launching i-pi...
bash /home/mm/repos/data/results/test_run/launch.sh
Launched! Please manually check if i-pi is running.
Starting clients with joblib...


wandb not installed, skipping import
fande.models module imported...
Icecream logger is not available
wandb not installed, skipping import
fande.models module imported...
Icecream logger is not available
wandb not installed, skipping import
fande.models module imported...
wandb not installed, skipping import
fande.models module imported...
Icecream logger is not available
Icecream logger is not available
Calculator is set up!
Launching client 1 at port 10200
wandb not installed, skipping import
fande.models module imported...
Icecream logger is not available
Calculator is set up!
Launching client 2 at port 10200
Calculator is set up!Calculator is set up!
Launching client 5 at port 10200

Launching client 3 at port 10200
wandb not installed, skipping import
fande.models module imported...
Icecream logger is not available


ConnectionRefusedError: [Errno 111] Connection refused

In [15]:
import os
os.getcwd()

'/home/mm/repos/rotor-gp/code'

In [12]:
ipi.exit_calculation(calc_dir)

0

In [36]:
ipi.restart(calc_dir)

0